# Excel Helper 

**Author:** 

In [1]:
import os, sys
import pandas as PD

In [2]:
MODULE_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(MODULE_PATH, 'data')
LOGIC_PATH = os.path.join(MODULE_PATH, 'logic')
sys.path.append(LOGIC_PATH)
print(MODULE_PATH)

C:\Users\pbusko\Projects\QualityCuration\base_module


In [3]:
import utility as UT
print([x for x in dir(UT) if '_' not in x and len(x) > 8])

['getNowStamp']


In [4]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import IPython.display as DS 
from ipywidgets import IntProgress

### Begin Code

In [5]:
# write to excel file

devPath = os.path.join(DATA_PATH, 'dev-workbook.xlsx')
writer = PD.ExcelWriter(devPath, engine='openpyxl')

devDf = PD.DataFrame([{'a': 1, 'b': 2}, {'a': 3, 'b': 4}, {'a': 5, 'b': 6}, ])
devDf.to_excel(writer, sheet_name='Sheet 1', index=False)

writer.close()

In [6]:
# read excel file

devPath = os.path.join(DATA_PATH, 'dev-workbook.xlsx')
excelFl = PD.ExcelFile(devPath)
sheetLs = excelFl.sheet_names
sheetDf = PD.read_excel(excelFl, 'Sheet 1')
excelFl.close()

sheetLs
sheetDf

['Sheet 1']

,a,b
0,1,2
1,3,4
2,5,6


In [7]:
# function to append data to an excel sheet

def appendToExcel(updateLs, sheetName):

    # check for the file

    reportPath = os.path.join(DATA_PATH, 'report.xlsx')
    if os.path.exists(reportPath) == False:
        writer = PD.ExcelWriter(reportPath, engine='openpyxl')
    else:
        writer = PD.ExcelWriter(reportPath, engine='openpyxl', mode='a')

    # check for the sheet

    sheetLs = [x.title for x in writer.book.worksheets]
    if sheetName not in sheetLs:
        reportDf = PD.DataFrame()
    else:
        excelFl = PD.ExcelFile(reportPath)
        reportDf = PD.read_excel(excelFl, sheetName, dtype=str)
        excelFl.close()

    # append data and overwrite excel

    updateDf = PD.DataFrame(updateLs, dtype=str)
    reportDf = PD.concat([reportDf, updateDf])
    writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)   # set sheets so it replaces correctly
    reportDf.to_excel(writer, sheet_name=sheetName, index=False)
    writer.close()

In [8]:
# function to delete a file

def resetReportFile():
    reportPath = os.path.join(DATA_PATH, 'report.xlsx')
    if os.path.exists(reportPath): os.remove(reportPath)

In [9]:
# function to delete a sheet in the file

def resetSheet(sheetName):
    reportPath = os.path.join(DATA_PATH, 'report.xlsx')
    try:
        writer = PD.ExcelWriter(reportPath, engine='openpyxl', mode='a')
        writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets) 
    except:
        print(f"no excel file found")
        return
    try:
        sheetRef = writer.book[sheetName]
        writer.book.remove(sheetRef)
        print("sheet removed")
    except:
        print(f"no sheet found: {sheetName}")
    writer.close()